In [2]:
import numpy as np
from transformer import Attention, Attention_cache, Attention_init

In [5]:
B, T, C = 2, 4, 6

In [3]:
rng = np.random.default_rng(2000)

``` python
@dataclass
class Attention_init:
    gamma: npt.NDArray[np.float64] # (1, 1, C) or (C, )
    beta: npt.NDArray[np.float64]  # (1, 1, C) or (C, )

    attentionHead: int             # H

    Wk: npt.NDArray[np.float64]    # (C, C) when C is hidden param or d_model
    Wq: npt.NDArray[np.float64]    # (C, C)
    Wv: npt.NDArray[np.float64]    # (C, C)

    merge_heads_bias: npt.NDArray[np.float64] # (1, 1, C) recommend

```

In [16]:
attentionInit = Attention_init(
    gamma=  rng.random((1, 1, C), dtype = np.float64),
    beta= rng.random((1, 1, C), dtype=np.float64),
    attentionHead= 2,
    Wk=rng.random((1, C, C), np.float64),
    Wq = rng.random((1, C, C), np.float64),
    Wv = rng.random((1, C, C), np.float64),
    merge_heads_bias=rng.random((1, 1, C), np.float64),

)

In [17]:
attention = Attention(attentionInit, "attentiondemo")

Exception: [Attention class] : shape of Wk is not allow (1, 6, 6)

In [18]:
def grad_check_transformer(forward, backward, eps=1e-6):
    rng = np.random.default_rng(1000)

    B, T, C = 2, 4, 6

    X = -rng.random((B, T, C), dtype=np.float64)
    G = -rng.random((B, T, C), dtype=np.float64) # upstream gradient

    # forward
    Y, cache = forward(X)

    # analytical gradient
    dX = backward(G, cache)

    # numerical gradient
    dX_num = np.zeros_like(X)
    it = np.nditer(X, flags=['multi_index'], op_flags=['readwrite'])

    while not it.finished:
        idx = it.multi_index
        old = X[idx]

        X[idx] = old + eps
        Yp, _ = forward(X)
        Lp = np.sum(Yp * G)

        X[idx] = old - eps
        Ym, _ = forward(X)
        Lm = np.sum(Ym * G)

        X[idx] = old
        dX_num[idx] = (Lp - Lm) / (2 * eps)

        it.iternext()

    # compare

    max_abs = np.max(np.abs(dX - dX_num))
    rel = max_abs / (np.max(np.abs(dX) + np.abs(dX_num)) + 1e-22)


    print("max_abs_diff:", max_abs)
    print("relative_diff:", rel)

In [19]:
grad_check_transformer(forward=attention.forward_train, backward=attention.backward, eps=1e-6)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 3)